In [ ]:
#Create Spark Session
from pyspark.sql import SparkSession
from cortex import configure_spark_with_cortex_pip

builder = SparkSession.builder \
            .appName("Streaming Datasource Example") \
            .master('local[*]') \
            .config("spark.ui.enabled", "false") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .config("spark.delta.logStore.gs.impl", "io.delta.storage.GCSLogStore") \
            .config("spark.jars.ivySettings", "ivy.settings") \
            .config("spark.cortex.proxy.impl", "com.c12e.cortex.profiles.intercept.TracingTimingMethodInterceptor")
spark = configure_spark_with_cortex_pip(builder).getOrCreate()

In [ ]:
from cortex.profiles.session import StorageOverrideBuilder
from cortex.profiles.session import CortexSession

sensaBuilder = CortexSession.sensaSessionBuilder(spark).standalone(
        catalogDir = "./spec",
        secretsJson = "{'local':{'secret-key':'myValue'}}",
        overrideStorageConfig = StorageOverrideBuilder.getInstance(spark).withFile("cortex-profiles", "../build/tmp/ingest")
    )

In [ ]:
#Create Cortex Session
from cortex.profiles.session import CortexSession
sensa = CortexSession.newSession(sparkSession=spark, options= sensaBuilder.buildWithSparkOverride())

In [ ]:
project = "local"
profile_schema_name = "member-profile"

In [ ]:
profileSchema = sensa.catalog().getProfileSchema(project, profile_schema_name)

In [ ]:
from py4j import java_gateway
from cortex.profiles.java import Supplier

job = sensa.job().ingestDataSource(project, profileSchema.getPrimarySource().getName(), sensa.getContext())
job.performFeatureCatalogCalculations(Supplier(lambda : True))
job.java.setQueryVariables(dict({}))
job.run()

In [ ]:
for join in profileSchema.getJoins(): 
    ingestJoin = sensa.job().ingestDataSource(project, join.getName(), sensa.getContext());
    ingestJoin.performFeatureCatalogCalculations(Supplier(lambda : True))
    ingestJoin.run()


In [ ]:
from py4j import java_gateway

buildProfileJob = sensa.job().buildProfile(project, profile_schema_name, sensa.getContext());
java_gateway.set_field(buildProfileJob.java, 'performFeatureCatalogCalculations', Supplier(lambda : True))
buildProfileJob.run()

In [ ]:
sensa.read().profile(project, profile_schema_name).load().toDF().show(10)